In [2]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import snscrape.modules.twitter as sntwitter
import matplotlib.pyplot as plt
import seaborn as sns
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
os.chdir('/Users/alisdghnia/Desktop/School - Graduate/Statistical Programming for Business Analytics - Spring 2023/')

In [43]:
df = pd.read_csv('chatGPT.csv')
df.head(6)

,Date,Tweet,Url,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,https://twitter.com/heyandriadaily/status/1619...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
1,2023-01-29 15:07:59+00:00,I've been waiting for ChatGPT to open back up...,https://twitter.com/_doc_oc/status/16197139759...,_doc_oc,2022-02-08 01:15:45+00:00,False,24,144,0,0,Texas Medical Center,☢️ B.S. Nuclear Engineering \n🩺⚙️ Texas A&M E...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,https://twitter.com/nikoonya/status/1619713889...,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,https://twitter.com/CSOonline/status/161971369...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
4,2023-01-29 15:06:18+00:00,AI trainers = underpaid human annotators?!\n#C...,https://twitter.com/OmaymaS_/status/1619713552...,OmaymaS_,2009-12-13 15:58:13+00:00,False,1199,786,0,0,NaN,#datascience #machinelearning #rstats #python...
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,https://twitter.com/KevinPetschow/status/16197...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...


In [46]:
# create a new column for the tweetID
df['tweetID'] = np.nan

# tweetID is the numbers after the last slash in the url
for i in range(len(df)):
    df['tweetID'][i] = df['Url'][i].split('/')[-1]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_8636/209259826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweetID'][i] = df['Url'][i].split('/')[-1]


In [91]:
data = []
for i, id in tqdm.tqdm(enumerate(df['tweetID']), total= len(df['tweetID'])):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
        
        data.append(new_data)

    if i > len(df['tweetID']):
        break

df2 = pd.DataFrame(data, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount', 
                                    'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

# add today's date to the dataframe
df2['date'] = pd.to_datetime('today').date()

  0%|          | 0/179252 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_8636/3019458125.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount]


In [92]:
df2

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339,214,0,1,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0
2,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3803,4933,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0
5,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [66]:
scraper = sntwitter.TwitterTweetScraper(tweetId = 1619713543825403904)
scraper

In [67]:
scraper

In [83]:
for i in scraper.get_items():
    break
print(i.lang)

en
